<a href="https://colab.research.google.com/github/hinsley/colabs/blob/master/TDt3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TDt3

Temporal distance model for tic-tac-toe

Inspired by Sutton & Barto

Plays as X

Board position encodings:

$\begin{bmatrix}0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8\end{bmatrix} \to \begin{pmatrix} 0 & 1 & 2 \\ 3 & 4 & 5 \\ 6 & 7 & 8 \end{pmatrix}$

In [0]:
import numpy as np
from typing import Dict, List, Optional, Tuple

In [0]:
ActionValue = Optional[float] # [0.0, 1.0]. Higher values are better. We
                              # consider draws to be losses.

PositionState = int # Enumeration. See below.
VALUE_BLANK = 0
VALUE_X = 1
VALUE_O = 2

BoardState = List[PositionState] # (3 x 3)

EncodedBoardState = int # Trinary encoding.

In [0]:
#@title eval_state(state: BoardState) -> ActionValue
def eval_state(state: BoardState) -> ActionValue:
  """
  Used to generate a fresh action value. Only evaluates win/lose/draw
  conditions, outputting a 0.5 action value if the state does not signify the
  end of a game.

  Because it is not possible in Tic-Tac-Toe to achieve a board state such that
  both players have three moves in a row [sic], we can stop evaluating once the
  first row [sic] is detected.

  Draws are considered losses.
  """
  
  for group in [
    [0,1,2], # Rows
    [3,4,5],
    [6,7,8],
    [0,3,6], # Columns
    [1,4,7],
    [2,5,8],
    [0,4,8], # Diagonals
    [2,4,6],
  ]:
    if state[group[0]] == state[group[1]] == state[group[2]]:
      try:
        return {
          VALUE_X: 1.0,
          VALUE_O: 0.0,
        }[state[group[0]]]
      except KeyError: # Row of blanks
        continue
  
  if any([position == VALUE_BLANK for position in state]):
    return 0.5 # Board contains an empty space and no win condition achieved
  else:
    return 0.0 # Draw

In [0]:
#@title encode_board(state: BoardState) -> EncodedBoardState
def encode_board(state: BoardState) -> EncodedBoardState:
  """
  Encode a trinary board state into a single hashable value. Smallest position
  is top left.
  """

  return sum([value * 3 ** i for i, value in enumerate(state)])

In [0]:
#@title decode_board(encoded_state: EncodedBoardState) -> BoardState
def decode_board(encoded_state: EncodedBoardState) -> BoardState:
  """
  Decode a single hashable representation of board state into an indexable
  serialization of positions.
  """

  return [encoded_state // (3 ** i) % 3 for i in range(9)]

In [0]:
#@title pprint(state: Optional[BoardState])
def pprint(state: BoardState):
  """ Pretty prints a board state. """
  
  if state is None:
    return

  def graphical_position_state(position_state: PositionState) -> str:
    return {
        VALUE_BLANK: "-",
        VALUE_X: "X",
        VALUE_O: "O",
    }[position_state]
  
  print(*[graphical_position_state(pstate) for pstate in state[:3]])
  print(*[graphical_position_state(pstate) for pstate in state[3:6]])
  print(*[graphical_position_state(pstate) for pstate in state[6:]])

In [0]:
class ActionEvaluator():

  _action_values: Dict[EncodedBoardState, ActionValue]

  def __init__(self):
    self._action_values = {0: 0.5}
  
  def evaluate(self, state: BoardState) -> ActionValue:
    encoded_state = encode_board(state)

    if not encoded_state in self._action_values: # Unexplored action.
      self._action_values[encoded_state] = eval_state(state)
    
    return self._action_values[encoded_state]

  def back_up_value(self,
                    prev_state: BoardState,
                    new_state: BoardState,
                    learning_rate=0.1):
    self._action_values[encode_board(prev_state)] += learning_rate * (self.evaluate(new_state) - self.evaluate(prev_state))

  def best_move(self, state: BoardState) -> BoardState:
    def possible_move_generator():
      for i, position_value in enumerate(state):
        if position_value == VALUE_BLANK:
          state_after_move = state.copy()
          state_after_move[i] = VALUE_X
          yield state_after_move
    possible_states = possible_move_generator()

    max_action_value, argmax = 0, None
    for new_state in possible_states:
      if argmax is None: # First move evaluated.
        argmax = new_state
      action_value = self.evaluate(new_state)
      if action_value > max_action_value: # Found a better move!
        max_action_value = action_value
        argmax = new_state
      if max_action_value == 1.0: # No point in looking for a better move.
        break

    # We never have to worry about returning `None` here, as that means we've
    # already reached a draw.

    return argmax

In [0]:
#@title move_o(action_evaluator: ActionEvaluator, state: BoardState, row: int, col: int) -> Optional[BoardState]
def move_o(action_evaluator: ActionEvaluator, state: BoardState, row: int, col: int) -> Optional[BoardState]:
  """ Makes a move for O in place. Row and col are zero-indexed. """
  
  if state[row * 3 + col] != VALUE_BLANK:
    print("That's not valid -- someone has already moved there!")
    return

  prev_state = state.copy()
  state[row * 3 + col] = VALUE_O
  action_evaluator.back_up_value(prev_state, state)

  if eval_state(state) == 0.0:
    print("O wins!")

  return state

In [0]:
#@title move_x(action_evaluator: ActionEvaluator, state: BoardState) -> BoardState
def move_x(action_evaluator: ActionEvaluator, state: BoardState) -> BoardState:
  """ Makes the best known move for X in place. """

  prev_state = state.copy()
  state[:] = action_evaluator.best_move(state)
  action_evaluator.back_up_value(prev_state, state)

  state_value = eval_state(state)
  if state_value == 1.0:
    print("X wins!")
  elif state_value == 0.0:
    print("Draw!")

  return state

In [260]:
#@title Turn 0: New game

try:
  initialized = initialized
except:
  print("Initializing ActionEvaluator")
  initialized = None
  action_evaluator = ActionEvaluator()

board_state = [VALUE_BLANK] * 9

pprint(board_state)

- - -
- - -
- - -


In [261]:
#@title Turn 1: X plays

pprint(move_x(action_evaluator, board_state))

X - -
- - -
- - -


In [262]:
#@title Turn 2: O plays

row =  1#@param
column =  1#@param

pprint(move_o(action_evaluator, board_state, row, column))

X - -
- O -
- - -


In [263]:
#@title Turn 3: X plays

pprint(move_x(action_evaluator, board_state))

X X -
- O -
- - -


In [264]:
#@title Turn 4: O plays

row =  0#@param
column =  2#@param

pprint(move_o(action_evaluator, board_state, row, column))

X X O
- O -
- - -


In [265]:
#@title Turn 5: X plays

pprint(move_x(action_evaluator, board_state))

X X O
- O -
X - -


In [266]:
#@title Turn 6: O plays

row =  1#@param
column =  0#@param

pprint(move_o(action_evaluator, board_state, row, column))

X X O
O O -
X - -


In [267]:
#@title Turn 7: X plays

pprint(move_x(action_evaluator, board_state))

X X O
O O -
X X -


In [268]:
#@title Turn 8: O plays

row =  2#@param
column =  2#@param

pprint(move_o(action_evaluator, board_state, row, column))

X X O
O O -
X X O


In [269]:
#@title Turn 9: X plays (Showdown!)

pprint(move_x(action_evaluator, board_state))

Draw!
X X O
O O X
X X O


In [271]:
#@title Run and copy the result of this cell to save/share your model's learned knowledge.
print(action_evaluator._action_values)

{0: 0.5, 1: 0.5, 3: 0.5, 9: 0.5, 27: 0.5, 81: 0.5, 243: 0.5, 729: 0.5, 2187: 0.5, 6561: 0.5, 163: 0.5, 166: 0.5, 172: 0.5, 190: 0.5, 406: 0.5, 892: 0.5, 2350: 0.5, 6724: 0.5, 184: 0.5, 211: 0.45, 427: 0.45, 913: 0.49995, 2371: 0.5, 6745: 0.5, 1669: 0.0, 1885: 0.0, 967: 0.49905, 1210: 0.486, 3154: 0.486, 7528: 0.495, 5584: 0.36450000000000005, 12145: 0.0, 16276: 0.36450000000000005, 16519: 0.0, 55: 0.5, 58: 0.5, 64: 0.5, 136: 0.5, 298: 0.5, 784: 0.5, 2242: 0.5, 6616: 0.5, 76: 0.5, 157: 0.5, 319: 0.5, 805: 0.5, 2263: 0.5, 6637: 0.5, 4531: 0.55, 4774: 0.5, 5260: 0.5, 11092: 1.0, 11902: 0.405}


# TODO: Rewrite the TD backup function to not require two games each time you want to back up a learned value.